In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

# load data

In [12]:
rowRdd = sc.textFile("/home/hsiehpinghan/git/python/spark-python-2/data/kaggle/stumbleupon/test.tsv")

In [18]:
headerRow = rowRdd.first()
headerRowColList = headerRow.replace("\"", "").split("\t")
headerRowColList

[u'url',
 u'urlid',
 u'boilerplate',
 u'alchemy_category',
 u'alchemy_category_score',
 u'avglinksize',
 u'commonlinkratio_1',
 u'commonlinkratio_2',
 u'commonlinkratio_3',
 u'commonlinkratio_4',
 u'compression_ratio',
 u'embed_ratio',
 u'framebased',
 u'frameTagRatio',
 u'hasDomainLink',
 u'html_ratio',
 u'image_ratio',
 u'is_news',
 u'lengthyLinkDomain',
 u'linkwordscore',
 u'news_front_page',
 u'non_markup_alphanum_characters',
 u'numberOfLinks',
 u'numwords_in_url',
 u'parametrizedLinkRatio',
 u'spelling_errors_ratio']

In [21]:
rowRddColList = rowRdd \
    .filter(lambda row : row != headerRow) \
    .map(lambda row : row.replace("\"", "")) \
    .map(lambda row : row.split("\t"))

# data cleaning

In [23]:
rowRddColList = rowRddColList \
    .map(lambda colList : \
        map(lambda col : None if col == "?" else col, colList)
    )
rowRddColList.take(1)

[[u'http://www.lynnskitchenadventures.com/2009/04/homemade-enchilada-sauce.html',
  u'5865',
  u'{title:Homemade Enchilada Sauce Lynn s Kitchen Adventures ,body:I usually buy my enchilada sauce Yes I knew I should be making it but I had never found a recipe that I was really happy with I had tried several and they just weren t very good So I stuck to the canned stuff you can get at the grocery store I was recently talking to a friend of mine about this She lived in Mexico for a few years so she knows some about Mexican cooking I asked her how she made her enchilada sauce She told me the basics and then gave me an exact recipe I decided to give it a try This recipe was really good This was the best enchilada sauce that I have made It had great flavor I think it was even better than the canned sauce My husband thought it could have been spicier But he likes his enchiladas spicy You can always add more chili powder or chilies if you like it really spicy The kids and I thought it was reall

# extract feature

In [24]:
alchemyCategoryMap = rowRddColList.map(lambda colList : colList[3]).distinct().zipWithIndex().collectAsMap()
alchemyCategoryMap

{None: 10,
 u'arts_entertainment': 11,
 u'business': 2,
 u'computer_internet': 3,
 u'culture_politics': 4,
 u'gaming': 0,
 u'health': 9,
 u'law_crime': 6,
 u'recreation': 1,
 u'religion': 8,
 u'science_technology': 5,
 u'sports': 7}

In [25]:
from pyspark.ml.feature import FeatureHasher

dataset = spark.createDataFrame([
    (2.2, True, "1", "foo"),
    (3.3, False, "2", "bar"),
    (4.4, False, "3", "baz"),
    (5.5, False, "4", "foo")
], ["real", "bool", "stringNum", "string"])

hasher = FeatureHasher(inputCols=["real", "bool", "stringNum", "string"],
                       outputCol="features")

featurized = hasher.transform(dataset)
featurized.show(truncate=False)

+----+-----+---------+------+--------------------------------------------------------+
|real|bool |stringNum|string|features                                                |
+----+-----+---------+------+--------------------------------------------------------+
|2.2 |true |1        |foo   |(262144,[174475,247670,257907,262126],[2.2,1.0,1.0,1.0])|
|3.3 |false|2        |bar   |(262144,[70644,89673,173866,174475],[1.0,1.0,1.0,3.3])  |
|4.4 |false|3        |baz   |(262144,[22406,70644,174475,187923],[1.0,1.0,4.4,1.0])  |
|5.5 |false|4        |foo   |(262144,[70644,101499,174475,257907],[1.0,1.0,5.5,1.0]) |
+----+-----+---------+------+--------------------------------------------------------+

